In [2]:
import numpy as np
import pandas as pd

In [4]:
df = pd.read_csv("../input/train.csv")

In [8]:
df.head()
data = df.values

In [21]:
y = data[:,0]
x = data[:,1:].astype(np.float64)

train_num = 41000
x_train, y_train  = x[:train_num], y[:train_num] 
x_val, y_val = x[train_num : ], y[train_num : ]

In [26]:
mean_pixel = x_train.mean(axis=0)
x_train -= mean_pixel
x_val -= mean_pixel

In [31]:
def initialize_global_weights():
    global W1, b1, W2, b2
    train_no, size = train_num, 784
    hidden, op = 500,10
    W1 = 0.001 * np.random.rand(size, hidden)
    b1 = np.zeros(hidden)
    W2 = 0.001 * np.random.rand(hidden, op)
    b2 = np.zeros(op)
initialize_global_weights()

In [38]:
def softmax(scores, y):
    N = scores.shape[0]
    scores = scores.copy()
    scores -= np.max(scores, axis=1)[:, None]
    probs = np.exp(scores)
    probs /= np.sum(probs, axis=1)[:, None]
    loss = np.sum(-np.log(probs[np.arange(N), y])) / N
    
    dscores = probs.copy()
    dscores[np.arange(N), y] -= 1
    
    return loss, dscores

In [46]:
def train(x, y= None, loss_fn = None, lr = 1e-3, reg = 0.0):
    global W1, W2,b1,b2
    a = x.dot(W1)+b1
    scores = a.dot(W2) + b2
    if y is None:
        return scores
    loss, dscores = loss_fn(scores, y)
    print('loss: %f' %loss)
    dW2 = np.dot(a.T, dscores) + reg * W2
    db2 = np.sum(dscores, axis=0)
    da = np.dot(dscores, W2.T)
    db1 = np.sum(da, axis=0)
    dW1 = np.dot(x.T, da) + reg * W1
    # update params
    W1 += - lr * dW1
    W2 += - lr * dW2
    b1 += - lr * db1
    b2 += - lr * db2
    return loss

In [47]:
num_iter = 10
initialize_global_weights()
for i in range(num_iter):
    loss = train(x_train, y_train, softmax, lr=1e-7, reg=1e-5)
    if np.isinf(loss):
        break

loss: 2.301413
loss: 2.895472
loss: 3.616603
loss: 2.953533
loss: 2.029714
loss: 1.731179
loss: 1.477431
loss: 1.246538
loss: 1.071168
loss: 0.942516


In [ ]:
train_score = train(x_train)
train_acc = (np.argmax(train_score, axis=1) == y_train).mean()
val_scores = train(x_val)
val_acc = (np.argmax(val_scores, axis=1) == y_val).mean() 
print(train_acc, val_acc)